In [1]:
import pandas as pd
import numpy as np
import requests
import json

import matplotlib.pyplot as plt
from scipy.stats import linregress

from pathlib import Path

## Importing Data

In [3]:
metacritic_path = Path("Resources/metacritic.csv")
metacritic_data = pd.read_csv(metacritic_path)
metacritic_data.head()

,id,name,year,metacritic_rating,reviewer_rating,positivity_ratio,to_beat_main,to_beat_extra,to_beat_completionist,extra_content_length,tags
0,96000,The Tiny Bang Story,2011.0,63.0,8.0,7.073879,3.60,3.60,3.77,0.17,Story Rich|Steampunk|Adventure|Atmospheric|Puz...
1,262410,World of Guns: Gun Disassembly,2014.0,NaN,8.0,5.208940,2.00,NaN,28.07,26.07,Horror|First-Person|Historical|Multiplayer|Str...
2,1250410,Microsoft Flight Simulator 40th Anniversary Ed...,2020.0,91.0,6.0,3.581082,NaN,NaN,NaN,NaN,Multiplayer|Adventure|VR|Action Roguelike|Phys...
3,365450,Hacknet,2015.0,82.0,8.0,14.548520,7.06,8.73,10.75,3.69,Horror|Story Rich|Hacking|Crime|Multiplayer|Dy...
4,92800,SpaceChem,2011.0,84.0,8.0,11.440415,43.32,57.79,67.55,24.23,Automation|Strategy|Building|Puzzle|Science|Pr...


In [6]:
metacritic_count = len(metacritic_data)
metacritic_count

63543

In [7]:
genre_path = Path("Resources/genre.csv")
genre_data = pd.read_csv(genre_path)
genre_data.head()

,Unnamed: 0,game,link,release,peak_players,positive_reviews,negative_reviews,total_reviews,rating,primary_genre,store_genres,publisher,developer,detected_technologies,store_asset_mod_time,review_percentage,players_right_now,24_hour_peak,all_time_peak,all_time_peak_date
0,0,Pizza Tower,/app/2231450/,2023-01-26,4529,19807,227,20034,96.39,Action (1),"Action (1), Indie (23)",Tour De Pizza,Tour De Pizza,Engine.GameMaker; SDK.FMOD,2023-01-10,98.0,"3,101","3,606",4529,2023-03-25
1,1,Resident Evil 4,/app/2050650/,2023-03-24,168191,61752,1616,63368,95.75,Action (1),"Action (1), Adventure (25)",CAPCOM Co. Ltd.,CAPCOM Co. Ltd.,Engine.RE_Engine; SDK.AMD_GPU_Services,2023-03-29,97.0,"46,289","47,164",168191,2023-03-25
2,2,The Murder of Sonic the Hedgehog,/app/2324650/,2023-03-31,15543,12643,213,12856,95.54,Casual (4),"Casual (4), Free to Play (37), Indie (23), Sim...",SEGA,SEGA,Engine.Unity; SDK.UnityBurst; SDK.cURL,2023-03-24,98.0,618,776,15543,2023-03-31
3,3,Pineapple on pizza,/app/2263010/,2023-03-28,1415,11717,209,11926,95.39,Indie (23),"Casual (4), Free to Play (37), Indie (23)",Majorariatto,Majorariatto,Engine.Unity; SDK.cURL,2023-03-22,98.0,101,101,1415,2023-03-28
4,4,Hi-Fi RUSH,/app/1817230/,2023-01-25,6132,14152,324,14476,95.09,Action (1),Action (1),Bethesda Softworks,Tango Gameworks,Engine.Unreal; SDK.Intel_OID; SDK.Intel_XeSS; ...,2023-04-11,97.0,630,664,6132,2023-01-29


In [8]:
genre_count = len(genre_data)
genre_count

67571

In [9]:
steamspy_tags_path = Path("Resources/steamspy_tags.csv")
steamspy_tags_data = pd.read_csv(steamspy_tags_path)
steamspy_tags_data.head()

,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,...,price,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count
0,Just Cause 2,2010-03-23,1,Avalanche Studios,Square Enix,windows,18,Single-player;Steam Achievements;Full controll...,Action;Adventure,Open World;Action;Sandbox,...,9.99,0.00,0.10,0.0,0.02,0.12,84.0,33.0,7.7,838.0
1,Call of Duty: World at War,2008-11-18,1,Treyarch,Activision,windows,16,Single-player;Multi-player;Co-op,Action,Zombies;World War II;FPS,...,19.99,0.02,0.14,0.0,0.03,0.18,83.0,36.0,7.5,721.0
2,Overlord II,2009-06-24,1,Triumph Studios;Virtual Programming,Codemasters,windows;mac;linux,0,Single-player,RPG,RPG;Villain Protagonist;Fantasy,...,6.99,0.00,0.03,0.0,0.00,0.03,79.0,32.0,8.1,159.0
3,Operation Flashpoint: Dragon Rising,2009-10-08,1,Codemasters Studios,Codemasters,windows,0,Single-player;Partial Controller Support,Action,Action;FPS;Tactical,...,6.99,0.02,0.03,0.0,0.01,0.05,76.0,29.0,4.9,539.0
4,F.E.A.R. 2: Project Origin,2009-02-12,1,Monolith,Warner Bros. Interactive Entertainment,windows,0,Single-player;Multi-player,Action,Horror;FPS;Action,...,9.99,0.02,0.05,0.0,0.01,0.07,79.0,49.0,7.8,520.0


In [10]:
steamspy_tags_count = len(steamspy_tags_data)
steamspy_tags_count

54